In [ ]:
snowflake_user='xxxx'
snowflake_password='xxx'
snowflake_account='xxxxx'

In [ ]:
from pytz import timezone
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import snowflake.connector
import logging
from datetime import datetime

In [ ]:
conf = snowflake.connector.connect(user=snowflake_user, password=snowflake_password, account=snowflake_account)
cursor = conf.cursor()
WAREHOUSE='FUSION_CS_DWH'
DATABASE="FUSION_FDR_DB"
SCHEMA="ANALYTICS"
cursor.execute("USE WAREHOUSE " + WAREHOUSE)
cursor.execute("USE DATABASE " + DATABASE)
cursor.execute("USE SCHEMA " + SCHEMA)

In [ ]:
merchant_list='CTDI'
OrgKey=(366,469,3488)
PB_list=('ATL3','CVG1','DFW4','EWR2','IND3','ONT1','ORD2','PHL2','RNO2','SEA1','SFO1','SLC1','ATL4','MCO1','BWI1','SCK1','LAX6')
event_list=['7','14','13','33']

In [ ]:
query = 'select case when packagekey is null then concat(smartlabelbarcode,\'_\') else \
concat(smartlabelbarcode,\'_\',packagekey) end as ID,\
a.*,fd.facilityname,fd2.facilityname from \
"FDR_DB"."FDR_SOURCE"."RETURNS_VOLUME_PREDICTION_DATA" a \
left join "FDR_DB"."FDR_SOURCE"."FACILITYDIM" fd \
on a.shiptofacilitykey=fd.facilitykey \
left join "FDR_DB"."FDR_SOURCE"."FACILITYDIM" fd2 \
on a.ORIGSHIPFROMFACILITYKEY=fd2.facilitykey \
where (a.destfacilityname=\'{0}\' or (MERCHANTORGKEY in {1} and (a.DESTFACILITYKEY in (\'1\')))) \
and trackingeventkey in {2} and a.eventdate is not null\
'.format(merchant_list,OrgKey,tuple(event_list))

In [ ]:
cursor.execute(query)

In [ ]:
df = cursor.fetchall()

In [ ]:

df = pd.DataFrame(df)
df.columns = ['ID','PackageKey','REVTRACKINGID','Smartlabelbarcode','TRACKINGNUMBER','MERCHANTORGKEY','DESTFACILITYKEY','DESTFACILITYNAME', \
'EVENTFACILITYKEY','EVENTFACILITYNAME','TRACKINGEVENTKEY','EventCode','DISPLAYTRACKINGEVENT','EVENTDATE','EVENTTIME',\
              'SHIPTOFACILITYKEY','ORIGSHIPFROMFACILITYKEY','EVENT','SHIPTOFACILITYNAME','ORIGSHIPFROMFACILITY']

In [ ]:
st=df[(df['DESTFACILITYKEY']==1)]
st=st[(~st['SHIPTOFACILITYNAME'].isin(PB_list)) & (~st['SHIPTOFACILITYNAME'].isnull())]
st=st[['ID','SHIPTOFACILITYNAME']]
st=st.drop_duplicates(['ID','SHIPTOFACILITYNAME'])

In [ ]:
df['DESTFACILITYNAME']=np.where(df['DESTFACILITYKEY']==1,np.nan,df['DESTFACILITYNAME'])
df['DESTFACILITYNAME']=df['DESTFACILITYNAME'].astype(str)
df2=df.merge(st[['ID','SHIPTOFACILITYNAME']],how='left',left_on='ID',right_on='ID')

In [ ]:
df2['DESTFACILITY2']=np.where((df2['DESTFACILITYKEY']==1) & (~df2['SHIPTOFACILITYNAME_y'].isnull()),df2['SHIPTOFACILITYNAME_y'],df2['DESTFACILITYNAME'])
df2['DESTFACILITY2']=df2['DESTFACILITY2'].astype(str)

In [ ]:
df2.drop(columns=['SHIPTOFACILITYNAME_y'],inplace=True)
df2.rename(columns = {'SHIPTOFACILITYNAME_x':'SHIPTOFACILITYNAME'}, inplace = True)

In [ ]:
rule=pd.read_excel('merchorgfile.xlsx')

In [ ]:
rule['IndFacility']=rule['IndFacility'].astype(str)

In [ ]:
df2['ORIGSHIPFROMFACILITY']=df2['ORIGSHIPFROMFACILITY'].astype(str)
rule['MerchOrg']=rule['MerchOrg'].astype(str)
df2['MERCHANTORGKEY']=df2['MERCHANTORGKEY'].astype(str)

In [ ]:
 for i in range(len(rule)):
        if rule.iloc[i][2]=='nan':
            df2['DESTFACILITY2']=np.where((df2['DESTFACILITY2']=='nan') & (df2['MERCHANTORGKEY']==rule.iloc[i][1]),rule.iloc[i][3],df2['DESTFACILITY2'])
        else:
            df2['DESTFACILITY2']=np.where((df2['DESTFACILITY2']=='nan') & (df2['MERCHANTORGKEY']==rule.iloc[i][1]) & (df2['ORIGSHIPFROMFACILITY'].isin(rule.iloc[i][2].split(","))),rule.iloc[i][3],df2['DESTFACILITY2'])

In [ ]:
df2=df2[df2['DESTFACILITY2']==merchant_list]

In [ ]:
df2.to_csv('atnt-ctdi.csv',index=False)

In [ ]:
inputFileName = 'input_atnt-ctdi_'+datetime.now().strftime('%Y%m%d_%H%M%S')+'.csv'

In [ ]:
df2['SNAPSHOT']=datetime.now().strftime('%Y-%m-%d')

In [ ]:
df2.to_csv(inputFileName,index=False,header=False)

In [ ]:
uploadToStageQuery = 'put file://'+inputFileName+' @%RVP_INPUT_VOLUME_DEV;'

In [ ]:
cursor.execute(uploadToStageQuery)

In [ ]:
copyIntoQuery = 'COPY INTO "RVP_INPUT_VOLUME_DEV" FROM @"%RVP_INPUT_VOLUME_DEV" FILE_FORMAT = "HIVEFORMAT"'

In [ ]:
cursor.execute(copyIntoQuery)

In [ ]:
cursor.close()